In [10]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

# Загрузка наборов данных
train_df = pd.read_parquet("train_data.pqt")
test_df = pd.read_parquet("test_data.pqt")

# Указание категориальных столбцов
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year",
]

# Преобразование столбцов в категориальный тип данных
train_df[cat_cols] = train_df[cat_cols].astype("category")
test_df[cat_cols] = test_df[cat_cols].astype("category")

# Определение признаков и целевой переменной
X = train_df.drop(["id", "date", "end_cluster"], axis=1)
y = train_df["end_cluster"]

# Разделение данных на тренировочный и валидационный наборы
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Кодирование целевой переменной
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Убедиться, что категориальные столбцы имеют тип данных "category"
for col in cat_cols:
    x_train[col] = x_train[col].astype('category')
    x_val[col] = x_val[col].astype('category')

# Обработка пропущенных значений
imputer = SimpleImputer(strategy='mean')
x_train_imputed = imputer.fit_transform(x_train.select_dtypes(include=[np.number]))
x_val_imputed = imputer.transform(x_val.select_dtypes(include=[np.number]))

# Применение PCA для уменьшения размерности
pca = PCA(n_components=0.999999999999999)  # Сохранение 99% дисперсии
x_train_pca = pca.fit_transform(x_train_imputed)
x_val_pca = pca.transform(x_val_imputed)

# Инициализация и обучение модели
model = XGBClassifier(random_state=42, n_jobs=-1, verbosity=0, enable_categorical=True)
model.fit(x_train_pca, y_train_encoded)

# Загрузка весов кластеров
cluster_weights = pd.read_excel("cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

# Предсказание вероятностей на валидационном наборе
y_pred_proba = model.predict_proba(x_val_pca)

# Определение функции расчета взвешенного ROC AUC
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels, multi_class="ovr", average=None)
    weighted_roc_auc = np.average(classes_roc_auc, weights=weights)
    return weighted_roc_auc

# Корректировка словаря весов для закодированных меток
corrected_weights_dict = {label_encoder.transform([k])[0]: v for k, v in weights_dict.items()}

# Расчет взвешенного ROC AUC
weighted_roc_auc_score = weighted_roc_auc(y_val_encoded, y_pred_proba, model.classes_, corrected_weights_dict)
print(weighted_roc_auc_score)

# Подготовка тестовых данных
test_df["start_cluster"] = train_df["start_cluster"].mode()[0]
test_df["start_cluster"] = test_df["start_cluster"].astype("category")

# Загрузка файла с примером отправки
sample_submission_df = pd.read_csv("sample_submission.csv")

# Выбор данных за последний месяц
last_m_test_df = test_df[test_df["date"] == "month_6"].drop(["id", "date"], axis=1)

# Обработка пропущенных значений в тестовых данных
last_m_test_imputed = imputer.transform(last_m_test_df.select_dtypes(include=[np.number]))

# Применение PCA к тестовым данным
last_m_test_pca = pca.transform(last_m_test_imputed)

# Предсказание вероятностей на тестовом наборе
test_pred_proba = model.predict_proba(last_m_test_pca)

# Преобразование предсказаний в DataFrame
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model.classes_)

# Сортировка столбцов
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

# Убедиться, что sample_submission_df имеет те же столбцы
sample_submission_df.iloc[:, 1:] = test_pred_proba_df.values

# Сохранение в CSV
sample_submission_df.to_csv("baseline.csv", index=False)


0.8724512482725781
